# Ejercicio 11 : Asistente RAG Conversacional

## Objetivo de la práctica

Construir un asistente que:

1. Recibe una pregunta del usuario
2. Recupera texto relevante de un corpus (ej. libro de Baeza-Yates)
3. Genera una respuesta basada en los documentos encontrados
4. Mantiene el historial de conversación


## Parte 0: Librerías necesarias
- openai
- faiss-cpu
- sentence-transformers

In [1]:
# Instalación de dependencias
#!pip install google-generativeai "faiss-cpu" "sentence-transformers" pymupdf

## Parte 1: Carga del corpus

Aquí se debe cargar el corpus con los documentos en PDF.

- Libro de Stanford
- Libro BM25
- Paper: Marcia Bates (1989). The design of browsing and berrypicking techniques for the online search interface

In [2]:
# Carga de los documentos PDF
import fitz  # PyMuPDF
import os

# Nombres de los archivos PDF que servirán como corpus
pdf_files = ["Libro_BM25.pdf", "Libro_Stanford.pdf", "Paper_Marcia_Bates.pdf"]

def load_corpus(file_paths):
    """
    Carga el texto y metadatos de una lista de archivos PDF.
    
    Args:
        file_paths (list): Lista de rutas a los archivos PDF.
        
    Returns:
        list: Una lista de diccionarios, cada uno con el nombre del archivo,
              el número de página y el contenido de esa página.
    """
    corpus_pages = []
    for path in file_paths:
        try:
            doc = fitz.open(path)
            for page_num, page in enumerate(doc):
                corpus_pages.append({
                    "source": os.path.basename(path),
                    "page": page_num + 1,
                    "content": page.get_text()
                })
            print(f"  - Procesado '{os.path.basename(path)}' con {len(doc)} páginas.")
        except Exception as e:
            print(f"Error procesando el archivo {path}: {e}")
            
    print(f"Carga finalizada. Total de páginas extraídas: {len(corpus_pages)}")
    return corpus_pages
corpus_documents = load_corpus(pdf_files)

# Imprimir un ejemplo de una página cargada
print("\nEjemplo de una página cargada:")
print(corpus_documents[10])

  - Procesado 'Libro_BM25.pdf' con 59 páginas.
  - Procesado 'Libro_Stanford.pdf' con 581 páginas.
  - Procesado 'Paper_Marcia_Bates.pdf' con 18 páginas.
Carga finalizada. Total de páginas extraídas: 658

Ejemplo de una página cargada:
{'source': 'Libro_BM25.pdf', 'page': 11, 'content': '2.4 Some Notation\n341\n2.4.2\nDetails\nWe start with the probability of relevance of a given document and\nquery.\nThe ﬁrst three steps are simple algebraic transformations. In\nStep (2.1), we simply replace the probability by an odds-ratio.2 In\nStep (2.2), we perform Bayesian inversions on both numerator and\ndenominator. In Step (2.3), we drop the second component which is\nindependent of the document, and therefore does not aﬀect the rank-\ning of the documents.\nIn Step (2.4) (term independence assumption), we expand each\nprobability as a product over the terms of the vocabulary. This step\ndepends on an assumption of statistical independence between terms —\nactually a pair of such assumptions,

## Parte 2: Procesamiento del Corpus

Aquí se debe obtener el corpus procesado. El corpus estará formado por documentos que corresponden a las secciones (o subsecciones) de los libros. Cada documento debe indicar a qué libro corresponde, así como las páginas en las que está dentro de ese libro.

Recuerden que los documentos procesados no deben contener textos o caracteres ajenos al tema del que tratan.  

In [3]:
# Configuración de NLTK y función de preprocesamiento
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
# Preprocesamiento del texto
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    """Limpia y normaliza el texto de forma menos agresiva."""
    # 1. Convertir a minúsculas
    text = text.lower()
    # 2. Eliminar solo caracteres especiales no deseados, pero conservar números, puntos y guiones.
    text = re.sub(r'[^a-z0-9\s\.\-]', '', text)
    # 3. Tokenizar
    tokens = word_tokenize(text)
    # 4. Eliminar stopwords y lematizar
    processed_tokens = [
        lemmatizer.lemmatize(word) for word in tokens if word not in stop_words
    ]
    return ' '.join(processed_tokens)
# Segmentación del corpus aplicando la limpieza
def process_and_chunk_corpus(documents, chunk_size=512, chunk_overlap=25):
    """
    Segmenta los documentos y aplica el preprocesamiento, guardando tanto el 
    chunk original como el procesado.
    """
    document_chunks = []
    for doc in documents:
        raw_content = doc["content"].replace('\n', ' ').strip()
        if not raw_content:
            continue
        start = 0
        while start < len(raw_content):
            end = start + chunk_size
            raw_chunk = raw_content[start:end]
            processed_chunk_text = preprocess_text(raw_chunk)
            
            if processed_chunk_text:
                document_chunks.append({
                    "source": doc["source"],
                    "page": doc["page"],
                    "raw_content": raw_chunk,
                    "processed_content": processed_chunk_text
                })
            start += chunk_size - chunk_overlap
            
    print(f"Segmentación finalizada. Número total de chunks: {len(document_chunks)}")
    return document_chunks

document_chunks = process_and_chunk_corpus(corpus_documents)

Segmentación finalizada. Número total de chunks: 3453


## Parte 3: Cálculo de Embeddings e Indexación en base de datos vectorial

Aquí, una vez que se ha calculado el embedding de cada documento, se deberá indexar este embedding en una base de datos vectorial como FAISS, ChromaDB o Pinecone

In [5]:
# Creación de embeddings e indexación
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

C:\Users\Christian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Generación de embeddings con SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
# Usamos el contenido preprocesado para generar los embeddings
corpus_embeddings = model.encode(
    [chunk['processed_content'] for chunk in document_chunks],
    convert_to_numpy=True,
    show_progress_bar=True
)
# Construir el índice FAISS
embedding_dim = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(corpus_embeddings)
print(f"Índice FAISS construido. Total de vectores indexados: {index.ntotal}")

Batches: 100%|██████████| 108/108 [00:36<00:00,  2.99it/s]

Índice FAISS construido. Total de vectores indexados: 3453


## Parte 4: Búsqueda y obtención del contexto

En esta parte debemos definir una _query_ y buscar los documentos que más se relacionan con ella.

Estos documentos formarán el contexto que vamos a entregar al LLM.

In [7]:
#Función de búsqueda semántica
def search_context(query, top_k=5):
    """
    Busca los chunks más relevantes para una query y retorna su contenido original.
    """
    print(f"\n Buscando contexto para la query: '{query}'")
    # Preprocesar la query del usuario de la misma manera que el corpus
    processed_query = preprocess_text(query)
    # Codificar la query procesada
    query_embedding = model.encode(processed_query, convert_to_numpy=True).reshape(1, -1)
    # Buscar en FAISS
    distances, indices = index.search(query_embedding, top_k)
    # Recuperar los chunks y formatear el contexto para el LLM
    context = ""
    retrieved_metadata = []
    for i, idx in enumerate(indices[0]):
        chunk = document_chunks[idx]
        context += f"Fuente {i+1} (Archivo: {chunk['source']}, Página: {chunk['page']}):\n"
        # Usamos el contenido original (raw) para el prompt del LLM
        context += f"\"{chunk['raw_content']}\"\n\n"
        retrieved_metadata.append(chunk)
    print("Contexto recuperado exitosamente")
    return context, retrieved_metadata

## Parte 5: Generación de Respuesta

Aquí, entregamos el contexto al LLM, y él nos responde a la _query_ con una explicación en lenguaje natural.

In [ ]:
# Generación de respuesta con Gemini y gestión de la conversación
import google.generativeai as genai
import getpass

GEMINI_API_KEY = "API"
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

In [9]:
# Mecanismo de mantenimiento conversacional "Contexto"
conversation_history = []
WINDOW_SIZE = 5

def generate_answer_with_chat(query):
    """Gestión de contexto y generación de la respuesta con Gemini"""
    global conversation_history
    
    history_str = "\n".join([f"Usuario: {q}\nAsistente: {a}" for q, a in conversation_history])
    contextualized_query = f"Historial de la conversación:\n{history_str}\nConsulta actual del usuario: {query}"
    
    context, metadata = search_context(contextualized_query, top_k=15)
    
    prompt = f"""
    *Rol y Objetivo:* Eres un asistente experto en Recuperación de Información. Tu objetivo no es solo encontrar información, sino sintetizar el conocimiento contenido en los fragmentos de texto proporcionados para dar una respuesta completa y bien fundamentada a la "Consulta actual del usuario".

    Instrucciones Clave:
    1.  *Analiza y Sintetiza: Lee y comprende todos los fragmentos en el "Contexto de documentos". No te limites a un solo fragmento. Conecta ideas de diferentes fuentes para construir la mejor respuesta posible.
    2.  *Infiere si es necesario: Si la respuesta no está explícitamente declarada pero puede ser razonablemente deducida a partir de la información disponible, hazlo.
    3.  *Cita tus fuentes: Tu respuesta DEBE estar fundamentada en el texto. Cita cada pieza de información que uses con el formato [Fuente X, Archivo: <nombre>, Página: <numero>].
    4.  *Fallback Inteligente: Si después de analizar profundamente todo el contexto, la información simplemente no existe para responder la pregunta, indica amablemente que el tema parece estar fuera del alcance del conocimiento de los documentos que componen el corpus. Responde "No tengo suficiente información para res´ponder la pregunta" si no existe ninguna relación de la consulta del usuario con el contenido recuperado.

    --- Contexto de documentos ---
    {context}
    --- Fin del Contexto ---
    
    {contextualized_query}
    
    **Asistente Experto:**
    """
    if not gemini_model:
        return "El servicio de GEMINI no está disponible", []

    try:
        response = gemini_model.generate_content(prompt)
        answer = response.text
        
        conversation_history.append((query, answer))
        if len(conversation_history) > WINDOW_SIZE:
            conversation_history.pop(0)
            
        return answer, metadata
    except Exception as e:
        return f"Error al generar la respuesta con Gemini: {e}", []

In [10]:

# --- Ciclo de conversación ---
print("\n\n--- INICIANDO ASISTENTE RAG CONVERSACIONAL CON GEMINI ---")
print("Escribe 'salir' para terminar la conversación.\n")

while True:
    user_query = input("\nUsuario: ")
    if user_query.lower() == 'salir':
        break
    
    final_answer, sources = generate_answer_with_chat(user_query)
    print("\n----------------")
    print("\nAsistente:")
    print(final_answer)
    print("\n----------------")



--- INICIANDO ASISTENTE RAG CONVERSACIONAL CON GEMINI ---
Escribe 'salir' para terminar la conversación.


 Buscando contexto para la query: 'Historial de la conversación:

Consulta actual del usuario: Quien es Marcia Bates?'
Contexto recuperado exitosamente

----------------

Asistente:
Marcia Bates es una autora que ha publicado trabajos en el campo de la recuperación de información y la bibliografía. Entre sus publicaciones se encuentran:

*   "The Fallacy of the Perfect 30-Item Online Search", publicada en RQ, 24, 1, 1984, pp. 43-50 [Fuente 10, Archivo: Paper_Marcia_Bates.pdf, Página: 17].
*   "Rigorous Systematic Bibliography", publicada en RQ, 16, 1, 1976, pp. 7-26 [Fuente 10, Archivo: Paper_Marcia_Bates.pdf, Página: 17].

----------------

 Buscando contexto para la query: 'Historial de la conversación:
Usuario: Quien es Marcia Bates?
Asistente: Marcia Bates es una autora que ha publicado trabajos en el campo de la recuperación de información y la bibliografía. Entre sus publi